In [4]:
#Generate the uniform random variables
import math
def Gen_Uniform (n,seed=1):
    result=[]
    for i in range(n):
        seed=(seed*39373)%(2**31-1)
        result.append(seed/(2**31-1))
    return result
    

In [5]:
#Generate the standard normal random variables
import matplotlib.pyplot as plt
def Gen_Normal (n,seed=1):
    u=Gen_Uniform(n,seed)
    result=[]
    a0=2.50662823884
    b0=-8.47351093090
    a1=-18.61500062529
    b1=23.08336743743
    a2=41.39119773534
    b2=-21.06224101826
    a3=-25.44106049637
    b3=3.13082909833
    c0=0.3374754822726147
    c5=0.0003951896511919
    c1=0.9761690190917186
    c6=0.0000321767881768
    c2=0.1607979714918209
    c7=0.0000002888167364
    c3=0.0276438810333863
    c8=0.0000003960315187
    c4=0.0038405729373609
    
    for ui in u:
        y=ui-0.5
        if abs(y)<0.42:
            r=y*y
            x=y*(((a3*r+a2)*r+a1)*r+a0)/((((b3*r+b2)*r+b1)*r+b0)*r+1)
        else:
            r=ui
            if y>0:
                r=1-ui
            r=math.log(-math.log(r))
            x=c0+r*(c1+r*(c2+r*(c3+r*(c4+r*(c5+r*(c6+r*(c7+r*c8)))))))
            if y<0:
                x=-x
        result.append(x)
    return result

#nm=Gen_Normal(100000)
#plt.hist(nm)

In [10]:
def Gen_Option_Data (n,S0=41,K=42,T=0.75,sig=0.25,q=0.01,r=0.03):
    C=[]
    Cplus=[]
    Cminus=[]
    DeltaC=[]
    VegaC=[]
    P=[]
    DeltaP=[]
    VegaP=[]
    nm=Gen_Normal(n)
    ds=0.01
    
    for zi in nm:
        Si=S0*math.exp((r-q-sig*sig/2)*T+sig*math.sqrt(T)*zi)
        Siplus=(S0+ds)*math.exp((r-q-sig*sig/2)*T+sig*math.sqrt(T)*zi)
        Siminus=(S0-ds)*math.exp((r-q-sig*sig/2)*T+sig*math.sqrt(T)*zi)
        Ci=math.exp(-r*T)*max(Si-K,0)
        Ciplus=math.exp(-r*T)*max(Siplus-K,0)
        Ciminus=math.exp(-r*T)*max(Siminus-K,0)
        C.append(Ci)
        Cplus.append(Ciplus)
        Cminus.append(Ciminus)
        Pi=math.exp(-r*T)*max(K-Si,0)
        P.append(Pi)
        if Si>K:
            DeltaCi=math.exp(-r*T)*Si/S0
            VegaCi=Si*math.exp(-r*T)*(-sig*T+math.sqrt(T)*zi)
        else:
            DeltaCi=0
            VegaCi=0
        DeltaC.append(DeltaCi)
        VegaC.append(VegaCi)
        if K>Si:
            DeltaPi=-math.exp(-r*T)*Si/S0
            VegaPi=-Si*math.exp(-r*T)*(-sig*T+math.sqrt(T)*zi)
        else:
            DeltaPi=0
            VegaPi=0
        DeltaP.append(DeltaPi)
        VegaP.append(VegaPi)
    C0=sum(C)/n
    C0plus=sum(Cplus)/n
    C0minus=sum(Cminus)/n
    result=[sum(C)/n,sum(DeltaC)/n,sum(VegaC)/n,sum(P)/n,sum(DeltaP)/n,sum(VegaP)/n,(C0plus-C0minus)/2/ds,(C0plus+C0minus-2*C0)/ds/ds]#C,DeltaC,VegaC,P,DeltaP,VegaP,fdDeltaC,fdGammaC
    return result

def Cal_Err (Opt_Data,n):
    accu=[3.341182879,0.5224707996,14.0286135192,3.7130845925,-0.4700572552,14.0286135192,0.5224707996,0.0448438585]
    result=[abs(accu[i]-Opt_Data[i])*math.sqrt(n) for i in range(8)]
    return result

for k in range(10):
    n=10000*(2**k)
    print("n=",n)
    Opt_Data=Gen_Option_Data(n)
    Error=Cal_Err(Opt_Data,n)
    print("Monte-Carlo:",Opt_Data,)
    print("Error:",Error)


n= 10000
Monte-Carlo: [3.3644432468283156, 0.5258680322254566, 14.120530414994844, 3.7071440414754884, -0.4673722401519318, 13.993430088434584, 0.5258801373623667, 0.040142107700091856]
Error: [2.326036782831542, 0.33972326254566587, 9.191689579484397, 0.5940551024511631, 0.2685015048068229, 3.518343076541619, 0.34093377623667376, 0.4701750799908144]
n= 20000
Monte-Carlo: [3.3470032796691047, 0.5201341933825695, 14.062738032656206, 3.738758993673281, -0.47190961754708877, 14.123050010739771, 0.5201133847420714, 0.031760190211116424]
Error: [0.8231289564693017, 0.3304460202615484, 4.825934973914945, 3.6309086345061403, 0.2619635953682159, 13.355336711847567, 0.33338880642224966, 1.850310113972994]
n= 40000
Monte-Carlo: [3.326600575457775, 0.5208384850445986, 13.965001132415473, 3.720117978334633, -0.4711623578637721, 14.05846728317545, 0.5208430226780747, 0.02853282141224156]
Error: [2.916460708445001, 0.3264629110802719, 12.722477356905415, 1.4066771669265954, 0.22102053275441236, 5.97